# CNKI论文摘要导出
## 使用说明
- 传入参数
  - search_expression(检索式),将要查询的检索式赋值给search_expression
  - download_path(默认下载地址),将想下载到的地址赋值给download_path,会修改浏览器默认下载位置
  - page_count(开始页数)，默认会从第page_count+1页位置处开始下载
- 目前的问题
  - 知网按检索式搜到3.9w条结果，但是界面上只展示了6000条，所以爬虫只爬了6k条。

In [22]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
# 配置检索式
search_expression = "(SU%= 口腔 OR CLC = 'R246.83' OR CLC = 'R322.41' OR CLC = 'R473.78' OR CLC = 'R529.8' OR CLC = 'R739.8' OR CLC = 'R78' OR CLC = 'R816.98' OR CLC = 'R828.8' OR CLC = 'R856.78' OR CLC = 'R988.2' OR CLC = 'TH787' OR CLC = 'S853.58' OR CLC = 'TQ658.4) OR (LY = 'International Journal of Oral science' OR LY = '中华口腔医学杂志' OR LY = '华西口腔医学杂志' OR LY = '临床口腔医学杂志' OR LY = '国际口腔医学杂志' OR LY = '中华老年口腔医学杂志' OR LY = '中国实用口腔科杂志' OR LY = '实用口腔医学杂志' OR LY = '口腔疾病防治' OR LY = '上海口腔医学' OR LY = '口腔医学研究' OR LY = '中华口腔正畸学杂志' OR LY = '口腔医学' OR LY = '口腔颌面修复学杂志' OR LY = '中国口腔种植学杂志' OR LY = '口腔材料器械杂志' OR LY = '中国口腔颌面外科杂志' OR LY = '北京口腔医学' OR LY = '口腔生物医学' OR LY = '口腔颌面外科杂志' OR LY = '现代口腔医学杂志' OR LY = '中华口腔医学研究杂志(电子版)' OR LY = '全科口腔医学电子杂志' OR LY = '牙体牙髓牙周病学杂志')"

# 设置默认论文下载路径
download_path = "D:\Desktop\论文自动化下载\CNKI论文摘要"
#开始页数
page_count=0
#论文检索开始终止时间
start_time="2015-01-01"
end_time="2015-03-10"
#proxy = "your_proxy_ip:port"
# 设置浏览器选项
options = Options()
options.add_argument("--start-maximized")
#options.add_argument(f'--proxy-server={proxy}')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 启动浏览器
browser = webdriver.Edge(options=options)

# 打开网页
browser.get("https://kns.cnki.net/kns8s/AdvSearch?crossids=YSTT4HG0%2CLSTPFY1C%2CJUP3MUPD%2CMPMFIG1A%2CWQ0UVIAA%2CBLZOG7CK%2CPWFIRAGL%2CEMRPGLPA%2CNLBO1Z6R%2CNN3FJMUV")

# 执行检索
professional_search_button = browser.find_element(By.XPATH, "//*[@id='ModuleSearch']/div[1]/div/div[2]/ul/li[3]")
professional_search_button.click()
text_input = browser.find_element(By.XPATH, "//*[@id='ModuleSearch']/div[1]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/textarea")
text_input.send_keys(search_expression)
# 使用JavaScript设置日期
date_input = browser.find_element(By.ID, "datebox0")
browser.execute_script("arguments[0].removeAttribute('readonly')", date_input)
date_input.clear()
date_input.send_keys(start_time)
date_input = browser.find_element(By.ID, "datebox1")
browser.execute_script("arguments[0].removeAttribute('readonly')", date_input)
date_input.clear()
date_input.send_keys(end_time)
search_button = browser.find_element(By.XPATH, "//*[@id='ModuleSearch']/div[1]/div/div[2]/div/div[1]/div[1]/div[2]/div[3]/input")
search_button.click()
# 等待下拉框可点击
dropdown = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="perPageDiv"]/div'))
)

browser.execute_script("arguments[0].click();", dropdown)

# 等待并选择特定的选项
option = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="perPageDiv"]/ul/li[3]'))
)
browser.execute_script("arguments[0].click();", option)
paper_count=0

for i in range(page_count):
    # 尝试点击“下一页”按钮
    next_button = WebDriverWait(browser, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='PageNext']"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
    browser.execute_script("arguments[0].click();", next_button)
    time.sleep(1)  # 等待页面加载
time.sleep(5)  # 等待页面加载
# 下载并翻页
flag=True
while flag:
    print("Downloading page", paper_count, "...")
    for i in range (1,11):
        select_all=browser.find_element(By.XPATH, "//*[@id='selectCheckAll1']")
        select_all.click()
        try:
            # 尝试点击“下一页”按钮
            next_button = WebDriverWait(browser, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='PageNext']"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
            time.sleep(3)  # 等待页面加载
        except Exception:
            flag=False
            break
    # 点击批量引用按钮
    export_button = browser.find_element(By.XPATH, "//*[@id='batchOpsBox']/li[2]/ul/li[1]/ul/li[6]")
    #export_button.click()
    browser.execute_script("arguments[0].click();", export_button)
    original_handles = browser.window_handles
    WebDriverWait(browser, 30).until(lambda d: len(d.window_handles) > 1)
    # 切换到新窗口
    browser.switch_to.window(original_handles[-1])
    time.sleep(5)  # 等待页面加载
    expport_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='litotxt']"))
    )
    try:
           ActionChains(browser).move_to_element(expport_button).click().perform()
    except Exception as e:
        print("Error clicking element:", e)
    time.sleep(1)  # 等待页面加载
    # 关闭当前窗口
    browser.close()

    # 切换回原始窗口
    browser.switch_to.window(original_handles[0])
    time.sleep(3)  # 等待页面加载
    clear_button = browser.find_element(By.XPATH, "//*[@id='briefBox']/div[1]/div/div[2]/div[1]/a")
    browser.execute_script("arguments[0].click();", clear_button)
    paper_count+=500
# 关闭浏览器
#browser.quit()


<>:13: SyntaxWarning: invalid escape sequence '\D'
<>:13: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_28656\1462574721.py:13: SyntaxWarning: invalid escape sequence '\D'
  download_path = "D:\Desktop\论文自动化下载\CNKI论文摘要"


# 下载摘要格式化脚本
## 使用说明
- 将"./CNKI摘要"目录下的所有txt文件解析并保存成格式化的CNKI_abstract.json

In [25]:
process_path = "./CNKI论文摘要"
#依次遍历目录下的文件
import os
import json
json_list = []
print("开始解析文件...")
for root, dirs, files in os.walk(process_path):
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
        # 解析文件内容
        abstracts=[]
        for i in range(0, len(lines), 2):
            if i + 2 < len(lines):
                entry =[lines[i].strip(), lines[i + 1].strip()]
                abstracts.append(entry)
        # 输出结果
        for abstract in abstracts:
            if len(abstract) < 2:
                continue
            reference = {
                "作者": abstract[0].split(".")[0].split("]")[1].strip() if len(abstract[0].split(".")[0].split("]"))>1 else "",
                "题目": abstract[0].split(".")[1].strip() if len(abstract[0].split("."))>1 else "",
                "期刊信息": abstract[0].split(".")[2].strip() if len(abstract[0].split("."))>2 else "",
                "摘要": abstract[1].strip().replace("摘要:", "")
            }
            json_list.append(reference)
            f.close()
# 将解析后的数据转换为 JSON 格式
json_data = json.dumps(json_list, ensure_ascii=False, indent=4)
print("总共包含", len(json_list), "条文献数据")
# 保存为文件
with open("./CNKI_abstract.json", "w", encoding="utf-8") as f:
    f.write(json_data)

print("JSON 数据已保存到 CNKI_abstract.json")

开始解析文件...
总共包含 103511 条文献数据
JSON 数据已保存到 CNKI_abstract.json
